# Library

In [981]:
import flodym as fd
from flodym import ExcelDimensionReader, DimensionDefinition
from flodym import ExcelParameterReader, ParameterDefinition
import flodym.export as fde
import numpy as np

## System definition

In [982]:
dimension_definitions = [
    fd.DimensionDefinition(letter="t", name="time", dtype=int), 
    fd.DimensionDefinition(letter="c", name="time_construction", dtype=int), 
    fd.DimensionDefinition(letter="b", name="building sector", dtype=str), 
    fd.DimensionDefinition(letter="s", name="scenario", dtype=str), 
    fd.DimensionDefinition(letter="p", name="product", dtype=str), 
    fd.DimensionDefinition(letter="w", name="waste", dtype=str), 
]

In [983]:
parameter_definitions = [
    fd.ParameterDefinition(name="building area per capita", dim_letters=("t","b","s")),  
    fd.ParameterDefinition(name="population", dim_letters=("t",)), 
    fd.ParameterDefinition(name="concrete mass per area", dim_letters=("c","b","p","s")), 
    fd.ParameterDefinition(name="product share", dim_letters=("t","b","p")), 
    fd.ParameterDefinition(name="stock product age cohort", dim_letters=("t","c","s","b","p")),
    fd.ParameterDefinition(name="product lifetimes", dim_letters=("c","b","s")),
    fd.ParameterDefinition(name="waste share", dim_letters=("t", "w")),
]

In [984]:
process_names = [
    "sysenv", 
    "use", 
    "waste"   
]

In [985]:
flow_definitions = [
    fd.FlowDefinition(from_process_name="sysenv", to_process_name="use", dim_letters=("c", "s", "b", "p")),
    fd.FlowDefinition(from_process_name="use", to_process_name="waste", dim_letters=("c", "s", "w")),
]

In [986]:
stock_definitions = [
    fd.StockDefinition(
        name="use",
        process="use",
        dim_letters=("c", "s", "b", "p"),
        time_letter="c",
        subclass=fd.StockDrivenDSM,
        lifetime_model_class=fd.NormalLifetime,
    ),
        fd.StockDefinition(
        name="waste",
        process="waste",
        dim_letters=("c", "s", "w"),
        time_letter="c",
        subclass=fd.SimpleFlowDrivenStock, # stock as the accumulation of inflow
    ),
]

In [987]:
mfa_definition = fd.MFADefinition(
    dimensions=dimension_definitions,
    parameters=parameter_definitions,
    processes=process_names,
    flows=flow_definitions,
    stocks=stock_definitions,
)

## Data import

In [988]:
dimension_file = "/Users/zhangyiwen/Documents/EMPA/Dds_school/Project/Data/dimension.xlsx"
dimension_files = {d.name: dimension_file for d in dimension_definitions}
dimension_sheets = {d.name: d.name for d in dimension_definitions}
reader = ExcelDimensionReader(
    dimension_files=dimension_files,
    dimension_sheets=dimension_sheets,
)

In [989]:
dims = reader.read_dimensions(dimension_definitions)

In [990]:
parameter_file = "/Users/zhangyiwen/Documents/EMPA/Dds_school/Project/Data/parameter.xlsx"
parameter_files = {p.name: parameter_file for p in parameter_definitions}
parameter_sheets = {p.name: p.name for p in parameter_definitions}
reader = ExcelParameterReader(
    parameter_files=parameter_files,
    parameter_sheets=parameter_sheets,
    allow_missing_values=True,
    allow_extra_values=True,
)

In [991]:
parameters = reader.read_parameters(parameter_definitions=parameter_definitions, dims=dims)

In [992]:
class ConcreteMFA(fd.MFASystem):
    def compute(self): 
        
        future_concrete_stock = self.parameters["population"] * self.parameters["building area per capita"] * self.parameters["concrete mass per area"]
        print(f'future_concrete_stock dims: {future_concrete_stock.dims}')
        print(type(future_concrete_stock.dims))
        # t,b,s,c,p
        future_concrete_stock_value = future_concrete_stock.values[:,:,:,0,:]
        future_concrete_stock_no_c = fd.FlodymArray(dims=future_concrete_stock.dims.get_subset(("t","b","s","p")), 
                                                    values=future_concrete_stock_value)

        # set the lifetime model parameters for stocks["use"]
        self.stocks["use"].lifetime_model.set_prms(
            mean=self.parameters["product lifetimes"],
            std=0.3*self.parameters["product lifetimes"], # assume std = 30% of mean
        )
        # build survival function for stocks["use"] using sf function (numpy array, ["c", "C", "s", "b", "p"]) 
        survival_historic_values = self.stocks["use"].lifetime_model.sf
        # print(type(survival_historic_values))
        # print(survival_historic_values.shape)

        # create FlodymArray survival_historic for survival function with correct dimensions
        c2 = fd.Dimension(name="time_construction2", letter="C", items=self.dims["c"].items)
        dims = fd.DimensionSet(dim_list=[c2] + self.stocks["use"].dims.dim_list)
        # as the stock composition is only available for 2015, slice the survival histroric to keep only C=2015
        survival_historic = fd.FlodymArray(values=survival_historic_values, dims=dims)[{"C": 2015}]


        # calculate the historical inflow from the stock composition and survival function for 2015
        # will get warning message for years later than 2015 where survival function = 0
        # will result in nan, but we will replace nan with 0 
        # this change will not affect the calculation as the stock after 2015 will be overwritten with predicted data later
        
        inflow_area_historic = self.parameters["stock product age cohort"][{"t": 2015}] / survival_historic
        inflow_area_historic_values = inflow_area_historic.values
        inflow_area_historic_values_where_nan = np.isnan(inflow_area_historic_values)
        inflow_area_historic_values[inflow_area_historic_values_where_nan] = 0
        
        inflow_area_historic_nonan = fd.FlodymArray(values=inflow_area_historic_values, dims=inflow_area_historic.dims)
        print(f'inflow_area_historic_nonan: {inflow_area_historic_nonan.dims}')

        # feed the historical inflow to an inflow-driven DSM
        dsm_area_historic = fd.InflowDrivenDSM(
            dims=inflow_area_historic_nonan.dims,
            lifetime_model=self.stocks["use"].lifetime_model,
            time_letter="c",
        )
        dsm_area_historic.inflow[...] = inflow_area_historic_nonan * self.parameters["concrete mass per area"]
        dsm_area_historic.compute()


        print(f'dsm_historic.stock: {dsm_area_historic.stock.dims}')

        # change the use stock with historical stock data from DSM
        self.stocks["use"].stock[...] = dsm_area_historic.stock * self.parameters["concrete mass per area"]
        
        # overwrite future development with predicted stock 
        self.stocks["use"].stock[{"c": self.dims["t"]}] = future_concrete_stock_no_c

        print(f'stock use stock dims: {self.stocks["use"].stock.dims}')
        use_stock_correct = self.stocks["use"].stock.values

        # print(f'future stock values: {self.stocks["use"].stock}')

        # compute use stock inflow and outflow
        self.stocks["use"].compute()

        total_outflow = self.stocks["use"].outflow

        # define envsys => use flow
        self.flows["sysenv => use"][...] = self.stocks["use"].inflow
        # print(total_outflow)
        self.flows["use => waste"][...] = total_outflow * self.parameters["waste share"]
        # print(f'dimensions use_outflow: {self.flows["use => waste"].dims}')
        # print(f'use_outflow values: {self.flows["use => waste"].values}')


In [993]:
ConcreteMFA_FR = ConcreteMFA.from_excel(
    definition=mfa_definition,
    dimension_files=dimension_files,
    parameter_files=parameter_files,
    dimension_sheets=dimension_sheets,
    parameter_sheets=parameter_sheets,
    allow_extra_parameter_values=True,
    allow_missing_parameter_values=True,
)

In [994]:
ConcreteMFA_FR.compute() #execute

flow_dict = {}
new_flow_columns = ['Year', 'Type_Code', 'Building_Type', 'Standard_Code',]

for f in ConcreteMFA_FR.flows.values():
    flow_name = f.name
    flow_df = f.to_df()
    flow_dict[flow_name] = flow_df
    print(f.name, "\n", f.to_df(), "\n")



future_concrete_stock dims: DimensionSet (t,b,s,c,p) with shape (86, 2, 2, 201, 37):
  't': 'time' with length 86
  'b': 'building sector' with length 2
  's': 'scenario' with length 2
  'c': 'time_construction' with length 201
  'p': 'product' with length 37
<class 'flodym.dimensions.DimensionSet'>
inflow_area_historic_nonan: DimensionSet (c,s,b,p) with shape (201, 2, 2, 37):
  'c': 'time_construction' with length 201
  's': 'scenario' with length 2
  'b': 'building sector' with length 2
  'p': 'product' with length 37
dsm_historic.stock: DimensionSet (c,s,b,p) with shape (201, 2, 2, 37):
  'c': 'time_construction' with length 201
  's': 'scenario' with length 2
  'b': 'building sector' with length 2
  'p': 'product' with length 37
stock use stock dims: DimensionSet (c,s,b,p) with shape (201, 2, 2, 37):
  'c': 'time_construction' with length 201
  's': 'scenario' with length 2
  'b': 'building sector' with length 2
  'p': 'product' with length 37
sysenv => use 
                       

/Users/zhangyiwen/anaconda3/lib/python3.11/site-packages/flodym/flodym_arrays.py:349: RuntimeWarning:

divide by zero encountered in divide



In [995]:
flow_dict.keys()
a = flow_dict['sysenv => use']
a

value
time_construction scenario building sector            product                             
1900              BAU      nonresidential buildings r SFH_non-standard                 0.0
                                                      SFH_standard                     0.0
                                                      SFH_efficient                    0.0
                                                      SFH_ZEB                          0.0
                                                      MFH_non-standard                 0.0
...                                                                                    ...
2100              CE       residential buildings      nonres_hotels_restaurants_ZEB    0.0
                                                      nonres_other_non_standard        0.0
                                                      nonres_other_standard            0.0
                                                      nonres_other_efficient           0.0
                                                      nonres_other_ZEB                 0.0

[29748 rows x 1 columns]

In [996]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

ConcreteMFA_FR.check_mass_balance()

INFO:root:Checking mass balance of ConcreteMFA object...


ValueError: Mass balance check failed for the following processes: sysenv (max error: 26990283.063122287), use (max error: 2294174060.3653836), waste (max error: 1160582171.714258)

In [997]:
fig = fde.PlotlySankeyPlotter(mfa=ConcreteMFA_FR, exclude_processes=[]).plot()
fig.show()

In [ ]:
print(type(ConcreteMFA_FR))
print(ConcreteMFA_FR)


<class '__main__.ConcreteMFA'>
dims=DimensionSet(dim_list=[Dimension(name='time', letter='t', items=[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100], dtype=<class 'int'>), Dimension(name='time_construction', letter='c', items=[1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,